In [19]:
##import libs
import numpy as np
import plotly
from plotly.subplots import make_subplots
import os
import matplotlib.pyplot as plt #to enable plotting within notebook
from nilearn import image as nimg
from nilearn import plotting 
from bids.layout import BIDSLayout
import bids
from matplotlib.pyplot import figure
import mpld3
import pandas as pd
from pathlib import Path   
import nibabel as nb 
import plotly.express as px
from nilearn.datasets import fetch_icbm152_brain_gm_mask
from nilearn.plotting import plot_roi
from nilearn.glm.first_level import make_first_level_design_matrix, run_glm
import seaborn as sns
from scipy.stats import norm
from nilearn.glm.contrasts import compute_contrast
from nilearn.plotting import plot_contrast_matrix
from nilearn.reporting import make_glm_report
from nilearn.interfaces.bids import save_glm_to_bids
from ordered_set import OrderedSet
from nilearn.glm.first_level import first_level_from_bids
import pickle
import os.path
import pathlib
import gc
from nilearn import image
from templateflow import api as tflow
from nilearn.maskers import NiftiMasker
import traceback
import logging
from nilearn.glm.first_level import FirstLevelModel
import nilearn as nl
from nilearn.interfaces.fmriprep import load_confounds
import json
from nilearn import surface
import hcp_utils as hcp
from nilearn.glm.contrasts import compute_fixed_effects


 # run 1stlevel analysis

In [20]:
##Set path to the data folder
os.chdir('/media/hcs-sci-psy-narun/OpenNEURO_adhd/ds002424-download/')
print("The working directory has been changed!")


The working directory has been changed!


In [21]:
## set path & get layout object for later use:
data_dir = '/media/hcs-sci-psy-narun/OpenNEURO_adhd/ds002424-download/'


 ### set major parameters

 #### change task_label accordingly

In [22]:
## set variable for each dir task = WM

# task_label = ['SLD', 'SSI', 'SSD', 'SLI', 'VLD', 'VSI', 'VSD', 'VLI']
space_label = "MNI152NLin2009cAsym"
derivatives_folder = "derivatives/fmriprep"
# direction = ['LR', 'RL']
#contrast_type = 'story-math'
ses = 'T1'

 ### load participants list

In [26]:

subject_ID_df  = pd.read_csv(data_dir + "valid_subj_list.csv", dtype=str, index_col=0)
subject_ID = list(subject_ID_df.iloc[:,0])
subject_ID = [str(x[4:]) for x in subject_ID]
# subject_ID.remove('911849')
subject_2 = subject_ID[1:2]


 ## functions

 ### function: extract desired confounds
 #### chooses confound columns & deletes dummy scans
 #### returns new confound dataframe & list of dummy scans to fix event file

In [27]:
## function: extract desired confounds
def manage_confounds(original_confs):


       cosine = original_confs.filter(regex='^cosine',axis=1)
       cosine_regs = list(cosine.columns)
       non_steady = original_confs.filter(regex='^non_steady_state',axis=1)
       a_comp_cor = ['c_comp_cor_00', 'c_comp_cor_01', 'c_comp_cor_02', 'c_comp_cor_03', 'c_comp_cor_04',
                      'w_comp_cor_00', 'w_comp_cor_01', 'w_comp_cor_02', 'w_comp_cor_03', 'w_comp_cor_04']
       movement_regs = ['rot_x', 'rot_x_derivative1','rot_y','rot_y_derivative1','rot_z','rot_z_derivative1',
                      'trans_x', 'trans_x_derivative1','trans_y', 'trans_y_derivative1','trans_z', 'trans_z_derivative1']

       desired_confs = a_comp_cor + cosine_regs + movement_regs


       if ~set(desired_confs).issubset(original_confs.columns):
              all_confs = original_confs.columns.values.tolist()
              mising_confs = list(set(desired_confs) - set(all_confs))
              final_confounds = list(OrderedSet(desired_confs) - OrderedSet(mising_confs))
       else:
              final_confounds = desired_confs
       
       #get dummy scan rows     
       temp = original_confs.isna().any(axis=1)
       na_ind = list(temp[temp].index)
       dummy = list(np.where(non_steady == 1)[0])
       dummy_scans = list(set(na_ind+dummy))
       confs_final = original_confs.drop(dummy_scans,axis=0,inplace=False)
       confs_final = confs_final.reset_index(drop=True)

       confs_final = confs_final.loc[:,final_confounds]
       confs_final.insert(confs_final.shape[1],'linear_trend', range(len(dummy_scans), original_confs.shape[0]))

       return confs_final, dummy_scans


 ### function: create & save plots
 #### plots contrasts maps for effect size and z-score

In [28]:
## function: create & save plots
def manage_plots(unparced_z, unparced_eff , sub_id, contrast_t, task_label, output_dir):
     surface_zs = nl.plotting.view_surf(hcp.mesh.inflated,
     hcp.cortex_data(unparced_z), 
     bg_map=hcp.mesh.sulc, cmap='bwr')
     surface_zs.save_as_html(output_dir + '/sub-%s_ses-%s_task-%s_contrast-%s_z_parcelated_plot.html' %(sub_id, ses, task_label, contrast_t))
     surface_eff = nl.plotting.view_surf(hcp.mesh.inflated,
     hcp.cortex_data(unparced_eff), 
     bg_map=hcp.mesh.sulc, cmap='bwr')
     surface_eff.save_as_html(output_dir + '/sub-%s_ses-%s_task-%s_contrast-%s_effect_parcelated_plot.html' %(sub_id, ses, task_label, contrast_t))
     plt.close('all')
     gc.collect()


 ### function: prepare events file
 #### delete faulty scans based on fmriprep confound file and fix timings

In [29]:
## function: prepare events file:
def prepare_events(ind_events,dummy_scans, t_r):
    n_dummy = len(dummy_scans)
    if n_dummy != 0:
        #print(n_dummy)
        new_zero = n_dummy * t_r
        ind_events['onset'] = ind_events['onset'] - new_zero
        invalid_row_ind = np.asarray(ind_events['onset'] < 0).nonzero()
        if len(invalid_row_ind[0]) != 0:
            del_row = []
            for i in invalid_row_ind[0]:
                # print(i)
                ii=i
                new_dur = ind_events.at[ii,'onset']+ind_events.at[ii,'duration']
                if new_dur > 0:
                    ind_events.at[ii,'onset'] = 0 
                    ind_events.at[ii,'duration'] = new_dur
                else:
                    del_row.append(ii)
            ind_events = ind_events.drop(del_row)  
    
    return  ind_events


In [30]:
def to_block(ind_events):

    ind_events = ind_events[['onset', 'duration', 'trial_type']]
    for i in range(len(ind_events) - 2):
        if ind_events.at[i, 'trial_type'] == 'no_response':
            ind_events.at[i, 'trial_type'] = ind_events.at[i + 3, 'trial_type']
    # print(ind_events)
    # Identify block starts and ends
    ind_events['next_trial_type'] = ind_events['trial_type'].shift(-1)
    ind_events['block_end'] = ind_events['trial_type'] != ind_events['next_trial_type']
    ind_events['block_start'] = ind_events['block_end'].shift(1, fill_value=True)

    # Compute block onsets and ends
    block_starts = ind_events[ind_events['block_start']].copy()
    block_ends = ind_events[ind_events['block_end']].copy()

    # Ensure the same index for merging
    block_starts.index = range(len(block_starts))
    block_ends.index = range(len(block_ends))

    # Merge to get block start and end times
    blocks = block_starts.copy()
    blocks['block_onset'] = block_starts['onset']
    blocks['block_end'] = block_ends['onset'] + block_ends['duration']

    # Compute the block duration
    blocks['block_duration'] = blocks['block_end'] - blocks['block_onset']

    # Select relevant columns
    block_df = blocks[['block_onset', 'block_duration', 'trial_type']]
    block_df.columns = ['onset', 'duration', 'trial_type']

    return block_df



 ### function: prepare contrast matrix:
 #### defines the desired contrast to be computed later

In [31]:
# function: prepare contrast matrix:
def prepare_contrasts(design_mat):
    #print(event_design_map.shape)
    contrast_matrix = np.eye(design_mat.shape[1])
    contrasts = {
    column: contrast_matrix[i]
    for i, column in enumerate(design_mat.columns)
    }
    contrasts = {
    'twoback':(
    contrasts['twoback']
    ),
    'oneback':(
    contrasts['oneback']
    ),
    'twoback-oneback': (
    0.5*contrasts['twoback']
    - 0.5*contrasts['oneback']
    )
    }
    return contrasts


 ### function: mask QC
 #### plots individualized, data based and fmriprep template masks for comparison

In [32]:
# function: Reshape, Parcellate 
def R_P (stat_map):
    ##parcelate full
    col = np.ones(len(stat_map))
    map_2d=np.stack((stat_map,col),axis=1)
    map_parc_data = np.transpose(map_2d)

    parced_map = hcp.parcellate(map_parc_data, hcp.mmp)
    unparced_map = hcp.unparcellate(parced_map[0], hcp.mmp)

    return parced_map, unparced_map


In [33]:
# function:  convert to cifti 
def convert_to_cifti (stat_map, axes):
    ##prepare save as cifti
    map = np.reshape(stat_map, (-1, stat_map.shape[0]))
    #save contrasts as cifti full
    scalar_axis_map = nb.cifti2.ScalarAxis(['stat-map'])  # Takes a list of names, one per row
    map_header = nb.Cifti2Header.from_axes([scalar_axis_map, axes[1]])
    map_img = nb.Cifti2Image(map, header = map_header, ) #this throws a warning since I don't have a header    
                
    return map_img


In [34]:
def combine_fixed_effects(contrast_arrays, variance_arrays):
    # Convert to numpy arrays for element-wise operations
    contrast_arrays = np.array(contrast_arrays)
    variance_arrays = np.array(variance_arrays)
    variance_arrays[variance_arrays == 0] = 1e-9
    # Calculate weights as inverse of variances
    weights = 1.0 / variance_arrays

    # Weighted sum of contrasts
    weighted_contrast_sum = np.sum(weights * contrast_arrays, axis=0)

    # Sum of weights
    sum_of_weights = np.sum(weights, axis=0)

    # Combined contrast
    combined_contrast = weighted_contrast_sum / sum_of_weights

    # Combined variance
    combined_variance = 1.0 / sum_of_weights

    return combined_contrast, combined_variance

 ### function: dir GLM and extract contrasts
 #### main function

In [37]:
## function: run GLM and extract contrasts
def surf_contrast(iter, sub_label, data_dir, output_dir, temp_img, masker):
    try:
        warnings.filterwarnings(action='ignore')
        # map_dict = {1: "oneback", 2:"twoback"}
        path = (output_dir+'sub-{}'.format(sub_label))
        #print(path)
        if not os.path.exists(path):
            os.mkdir(path)
        #plt.ioff()


        # set up lists for averaging across direction
        z_data = []
        eff_data = []
        v_data = []
        print('start:'+sub_label)

        task_labels = ['SLD', 'SSI', 'SSD', 'SLI']# verbal = ['VLD', 'VSI', 'VSD', 'VLI']
        for task_label in task_labels:
            json_file = data_dir + 'derivatives/fmriprep/sub-%s/ses-T1/func/sub-%s_ses-T1_task-%s_space-fsLR_den-91k_bold.json' %(sub_label,sub_label, task_label)
            if os.path.isfile(json_file):
                with open(json_file, 'r') as f:
                    t_r = json.load(f)['RepetitionTime']
                
                fmriprep_bold = (data_dir + 'derivatives/fmriprep/sub-%s/ses-T1/func/sub-%s_ses-T1_task-%s_space-fsLR_den-91k_bold.dtseries.nii' %(sub_label, sub_label, task_label))
                eve_dir = (data_dir + 'sub-%s/ses-T1/func/sub-%s_ses-T1_task-%s_events.tsv' %(sub_label, sub_label, task_label))
                conf_dir = (data_dir + 'derivatives/fmriprep/sub-%s/ses-T1/func/sub-%s_ses-T1_task-%s_desc-confounds_timeseries.tsv' %(sub_label, sub_label, task_label))

                events = pd.read_csv(eve_dir , sep='\t')
                confounds = pd.read_csv(conf_dir, sep='\t')
                #func_mni_img = nimg.load_img(fmriprep_bold)

                # call confounds function:
                [confounds_final, dummy_scans] = manage_confounds(confounds)

                # confounds_final = confounds_final.reset_index(drop=True)

                # calculate frametimes using confounds file n_rows and T_R
                frame_times = (np.arange(confounds_final.shape[0]) * t_r)# + (t_r/2)

                # call prepare event file function
                ind_events = events.copy()
                # fix events names
                ind_events = ind_events.replace(regex=['1-'],value='one')
                ind_events = ind_events.replace(regex=['2-'],value='two')
                # ind_events['trial_type']= ind_events['trial_type'].map(map_dict)
                # print(ind_events.loc[:,'trial_type'])
                ind_events = to_block(ind_events)

                ind_events = prepare_events(ind_events, dummy_scans, t_r)
                # print(ind_events.loc[:,'trial_type'])
                # ind_events = ind_events.reset_index(drop=True)



                # save events file
                ind_events.to_csv(output_dir + 
                                'sub-%s/sub-%s_ses-%s_task-%s_events.tsv' %(sub_label, sub_label, ses, task_label), sep='\t')
                confounds_final.to_csv(output_dir + 
                                'sub-%s/sub-%s_ses-%s_task-%s_confs.tsv' %(sub_label, sub_label, ses, task_label), sep='\t')
                # smoothed_img = image.smooth_img(ind_image, 6)

                design_matrix = make_first_level_design_matrix(frame_times = frame_times, 
                                                                    events=ind_events, hrf_model='spm', 
                                                                    add_regs=confounds_final,
                                                                    drift_model= None, #drift_order=2, #high_pass=1/128, 
                                                                    # #fir_delays=[0], #add_reg_names=None, #min_onset=-24, #oversampling=50
                )
                # # add the linear trend column to design matrix (to match it with xcp-d)
                # end_col = design_matrix.shape[1]
                # design_matrix.insert(end_col-1, 'linear_trend', range(len(dummy_scans), confounds.shape[0]))
                # design_matrix.add_poly(order=1, include_lower=True)
                # design_matrix.add_poly(order=2, include_lower=True)

                #design_matrix.to_csv(data_dir + 'derivatives/nilearn_glm/sub-%s_task-%s_dir-%s_design' % (model.subject_label, task_label, direction[dir]))
                del confounds, events

                # load image file and remove dummy scans:

                cifti = nb.load(fmriprep_bold)
                cifti_data = cifti.get_fdata()
                cifti_hdr = cifti.header
                nifti_hdr = cifti.nifti_header
                axes = [cifti_hdr.get_axis(i) for i in range(cifti.ndim)]
                cifti_data = cifti_data[len(dummy_scans):,:]
                del cifti
                cifti_data = nl.signal.clean(cifti_data, detrend=False, standardize='zscore_sample', confounds=None, standardize_confounds=False,
                                            filter=False, low_pass=None, high_pass=None, t_r=t_r, ensure_finite=False )
                # fit the model 
                labels, estimates = run_glm(cifti_data, design_matrix.values, noise_model='ols')
                
                design_plot = plotting.plot_design_matrix(design_matrix)
                design_plot.figure.savefig(path + '/sub-%s_ses-%s_task-%s_design.svg' %(sub_label, ses, task_label))
                design_matrix.to_csv(path + '/sub-%s_ses-%s_task-%s_design.tsv' %(sub_label, ses, task_label), sep='\t')
                
                del cifti_data
                # call contrast matrix function:
                contrasts = prepare_contrasts(design_matrix)
                #print("contrast created")
                # compute and generate contrasts (betas and z score)
                for index, (contrast_id, contrast_val) in enumerate(contrasts.items()):
                    contrast_map = compute_contrast(labels, estimates, contrast_val, stat_type='t')
                    # We present the Z-transform of the t map.
                    zmap = contrast_map.z_score()
                    effect_size = contrast_map.effect_size()[0]
                    variance = contrast_map.effect_variance()


                    #plot contrast design
                    contrast_plot = plot_contrast_matrix(
                    contrast_val,
                    design_matrix,
                    colorbar=True,
                    )
                    contrast_plot.set_xlabel(contrast_id)
                    contrast_plot.figure.set_figheight(2)
                    contrast_plot.figure.set_tight_layout(True)
                    contrast_plot.figure.savefig(path + '/sub-%s_ses-%s_task-%s_contrast-%s_design.svg' %(sub_label, ses, task_label, contrast_id))
                    
                    parced_z, unparced_z = R_P(zmap)
                    parced_eff, unparced_eff = R_P(zmap)

                    headers = list(hcp.mmp.labels.values())[1:]
                    pd.DataFrame(parced_z, columns = headers).to_csv(path + '/sub-%s_ses-%s_task-%s_contrast-%s_z_parcelations.tsv' %(sub_label, ses, task_label, contrast_id), sep='\t')
                    pd.DataFrame(parced_eff, columns = headers).to_csv(path + '/sub-%s_ses-%s_task-%s_contrast-%s_effect_parcelations.tsv' %(sub_label, ses, task_label, contrast_id), sep='\t')

                    var_img = convert_to_cifti(variance, axes)

                    z_img = convert_to_cifti(zmap, axes)
                    z_img.to_filename(path + '/sub-%s_ses-%s_task-%s_contrast-%s_space-fsLR_den-91k_stat-z.dtseries.nii' %(sub_label, ses, task_label, contrast_id))
                    eff_img = convert_to_cifti(effect_size, axes)
                    eff_img.to_filename(path + '/sub-%s_ses-%s_task-%s_contrast-%s_space-fsLR_den-91k_stat-effect.dtseries.nii' %(sub_label, ses, task_label, contrast_id))
                    unp_z_img = convert_to_cifti(unparced_z, axes)
                    unp_z_img.to_filename(path + '/sub-%s_ses-%s_task-%s_contrast-%s_space-fsLR_den-91k_stat-z_parcellated.dtseries.nii' %(sub_label, ses, task_label, contrast_id))
                    unp_eff_img = convert_to_cifti(unparced_eff, axes)
                    unp_eff_img.to_filename(path + '/sub-%s_ses-%s_task-%s_contrast-%s_space-fsLR_den-91k_stat-effect_parcellated.dtseries.nii' %(sub_label, ses, task_label, contrast_id))
                    v_data.append(variance)
                    z_data.append(zmap)
                    eff_data.append(effect_size)
                    # plot contrast maps for each direction
                    # manage_plots(unparced_z, unparced_eff, sub_label, contrast_id, task_label, path)

                    #print("plots created")

                    plt.close('all')

        plt.close('all')

        r4= int(len(z_data)/4)
        dirM = 'spatial' # verbal 

        for ind , contr_id in enumerate(contrasts.keys()):
            # combine directions and calculate mean 

            # Compute fixed effects
            mean_z_arr, var_map_z = combine_fixed_effects([z_data[ind],z_data[ind+r4],z_data[ind+r4*2],z_data[ind+r4*3]], [v_data[ind],v_data[ind+r4],v_data[ind+r4*2],v_data[ind+r4*3]])  
            mean_effect_arr, var_map_eff = combine_fixed_effects([eff_data[ind],eff_data[ind+r4],eff_data[ind+r4*2],eff_data[ind+r4*3]], [v_data[ind],v_data[ind+r4],v_data[ind+r4*2],v_data[ind+r4*3]])   
            parced_Mz, unparced_Mz = R_P(mean_z_arr)
            parced_Meff, unparced_Meff = R_P(mean_effect_arr)

            pd.DataFrame(parced_Mz, columns = headers).to_csv(path + '/sub-%s_task-%s_contrast-%s_z_parcelations.tsv' %(sub_label, dirM, contr_id), sep='\t')
            pd.DataFrame(parced_Meff, columns = headers).to_csv(path + '/sub-%s_task-%s_contrast-%s_effect_parcelations.tsv' %(sub_label, dirM, contr_id), sep='\t')

            #print("mean created")
            mean_z_img = convert_to_cifti(mean_z_arr, axes)
            mean_z_img.to_filename(path + '/sub-%s_task-%s_contrast-%s_space-fsLR_den-91k_stat-z.dtseries.nii' %(sub_label, dirM, contr_id))
            mean_eff_img = convert_to_cifti(mean_effect_arr, axes)
            mean_eff_img.to_filename(path + '/sub-%s_task-%s_contrast-%s_space-fsLR_den-91k_stat-effect.dtseries.nii' %(sub_label, dirM, contr_id))

            unp_Mz_img = convert_to_cifti(unparced_Mz, axes)
            unp_Mz_img.to_filename(path + '/sub-%s_task-%s_contrast-%s_space-fsLR_den-91k_stat-z_parcellated.dtseries.nii' %(sub_label, dirM, contr_id))
            unp_eff_img = convert_to_cifti(unparced_Meff, axes)
            unp_eff_img.to_filename(path + '/sub-%s_task-%s_contrast-%s_space-fsLR_den-91k_stat-effect_parcellated.dtseries.nii' %(sub_label, dirM, contr_id))

            manage_plots(unparced_Mz, unparced_Meff, sub_label, contr_id, dirM, path)
        #     plt.close('all')

        print('done:'+sub_label)
        
        gc.collect()
    except Exception as e:
        #error.append(sub_label)
        logging.error(traceback.format_exc())
    # except ValueError:
    #     print(model.subject_label +'raised value error')
    # except TypeError:
    #     print(model.subject_label +'raised type error')
    # except MemoryError:
    #     print(model.subject_label +'raised memory error')    




 ### run in parallel for all subjects

In [38]:
## run parallel
import warnings
import joblib
from joblib import Parallel, delayed
from joblib import parallel_backend
from joblib import Memory

from joblib import Memory
warnings.filterwarnings(action='ignore')

#plt.ioff()
output_dir = data_dir + "derivatives/nilearn_glm/surf_WM_std/"
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

Parallel(n_jobs=30)(delayed(surf_contrast)(iter, subject_label, data_dir, output_dir,  mni_gm , masker) for iter, subject_label in enumerate(subject_ID))



pixdim[1,2,3] should be non-zero; setting 0 dims to 1


start:01


pixdim[1,2,3] should be non-zero; setting 0 dims to 1
pixdim[1,2,3] should be non-zero; setting 0 dims to 1
pixdim[1,2,3] should be non-zero; setting 0 dims to 1
pixdim[1,2,3] should be non-zero; setting 0 dims to 1


start:02
start:05
start:04
start:08


pixdim[1,2,3] should be non-zero; setting 0 dims to 1
pixdim[1,2,3] should be non-zero; setting 0 dims to 1
ERROR:root:Traceback (most recent call last):
  File "/tmp/ipykernel_10757/2240966615.py", line 154, in surf_contrast
UnboundLocalError: cannot access local variable 'contrasts' where it is not associated with a value

pixdim[1,2,3] should be non-zero; setting 0 dims to 1
pixdim[1,2,3] should be non-zero; setting 0 dims to 1
pixdim[1,2,3] should be non-zero; setting 0 dims to 1
pixdim[1,2,3] should be non-zero; setting 0 dims to 1
pixdim[1,2,3] should be non-zero; setting 0 dims to 1
pixdim[1,2,3] should be non-zero; setting 0 dims to 1
pixdim[1,2,3] should be non-zero; setting 0 dims to 1
pixdim[1,2,3] should be non-zero; setting 0 dims to 1
pixdim[1,2,3] should be non-zero; setting 0 dims to 1


...

ERROR:root:Traceback (most recent call last):
  File "/tmp/ipykernel_10757/2240966615.py", line 154, in surf_contrast
UnboundLocalError: cannot access local variable 'contrasts' where it is not associated with a value

ERROR:root:Traceback (most recent call last):
  File "/tmp/ipykernel_10757/2240966615.py", line 154, in surf_contrast
UnboundLocalError: cannot access local variable 'contrasts' where it is not associated with a value

ERROR:root:Traceback (most recent call last):
  File "/tmp/ipykernel_10757/2240966615.py", line 154, in surf_contrast
UnboundLocalError: cannot access local variable 'contrasts' where it is not associated with a value



start:68
done:19
start:69
start:70
start:71


ERROR:root:Traceback (most recent call last):
  File "/tmp/ipykernel_10757/2240966615.py", line 154, in surf_contrast
UnboundLocalError: cannot access local variable 'contrasts' where it is not associated with a value



done:12
start:72
done:35
start:73
done:50
start:74
done:36
start:75
done:59
done:52
start:76
start:77
done:37
start:78
done:38
start:79
done:41
done:39
done:47
done:46
done:51
done:48
done:49
done:57
done:40
done:60
done:44
done:58
done:54
done:67
done:68
done:56
done:53
done:61
done:69
done:71
done:66
done:63
done:73
done:72
done:75
done:74
done:77
done:76
done:78
done:79


[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

 ### get completed and incompleted and erroneous subjects

In [18]:
output_dir = output_dir

incomplete = []
nonexist = []
folder_is = []
for id in subject_ID:
    #print(id)
    diris = os.path.isdir(output_dir + 'sub-%s' %id)
    folder_is.append(diris)
    if diris == False:
        nonexist.append(id)
    else:
        completed = os.path.isfile(output_dir +'sub-%s/sub-%s_task-WM_dir-mean_contrast-twobk-zerobk_space-fsLR_den-91k_stat-effect_parcellated.dtseries.nii' %(id , id))
        if completed == False:
            incomplete.append(id)




NameError: name 'output_dir' is not defined

 ### number of :
 #### completed , all, missing file , incompltete

In [ ]:
print(len(subject_ID), len(nonexist), len(incomplete))



 ### save lists of completed and error subjects

In [ ]:
pd.DataFrame(nonexist).to_csv(output_dir + 'error_subjects')
pd.DataFrame(incomplete).to_csv(output_dir + 'incompleted_subjects')


